In [10]:
import pandas as pd
import random

In [11]:
# Import data, retrieve only coordinate information

# data = pd.read_csv('House/csv/random/house_random.csv', sep=',')[['X', 'Y', 'Z']]
data = pd.read_csv('House/csv/spatial/house_spatial.csv', sep=',')[['X', 'Y', 'Z']]

In [12]:
# Function that creates unique id

node_ids = set()
def generate_node_id(node_ids):
    new_id = random.randint(1, 1e5)
    while new_id in node_ids:
        new_id = random.randint(1, 1e5)
    node_ids.add(new_id)
    return new_id

In [13]:
# Node structure for k-d tree

class Node:
    def __init__(self, point, id, depth, points_count, left=None, right=None):
        self.point = point
        self.id = id
        self.depth = depth
        self.points_count = points_count
        self.left = left
        self.right = right

In [14]:
# Adjust the build_kdtree function to limit depth and include points count

def build_kdtree(points, depth=0, max_depth=4):
    if not points or depth >= max_depth:
        return None

    k = len(points[0])
    axis = depth % k
    points.sort(key=lambda x: x[axis])
    median = len(points) // 2
    node_id = generate_node_id(node_ids)

    return Node(
        point=points[median],
        id=node_id,
        depth=depth,
        points_count=len(points),
        left=build_kdtree(points[:median], depth + 1, max_depth),
        right=build_kdtree(points[median + 1:], depth + 1, max_depth)
    )

In [15]:
# Function to build k-d tree and output CSV files

def traverse_and_collect_data(node, depth=0, max_depth=4):
    if node is None or depth >= max_depth:
        return [], []

    nodes = [{'Id': node.id, 'Degree': node.points_count, 'Depth': node.depth}]
    edges = []

    if node.left is not None:
        edges.append({'Source': node.id, 'Target': node.left.id, 'Type': 'Directed'})
        left_nodes, left_edges = traverse_and_collect_data(node.left, depth + 1, max_depth)
        nodes += left_nodes
        edges += left_edges

    if node.right is not None:
        edges.append({'Source': node.id, 'Target': node.right.id, 'Type': 'Directed'})
        right_nodes, right_edges = traverse_and_collect_data(node.right, depth + 1, max_depth)
        nodes += right_nodes
        edges += right_edges

    return nodes, edges

In [16]:
points_with_id = data.assign(Id=data.index).values.tolist()

kdtree = build_kdtree(points_with_id)

nodes_data, edges_data = traverse_and_collect_data(kdtree)

In [17]:
nodes_df = pd.DataFrame(nodes_data)
edges_df = pd.DataFrame(edges_data)

# nodes_df.to_csv('House/csv/random/node.csv', index=False)
# edges_df.to_csv('House/csv/random/edge.csv', index=False)

nodes_df.to_csv('House/csv/spatial/node.csv', index=False)
edges_df.to_csv('House/csv/spatial/edge.csv', index=False)